# GRIK Test

## 1. Import 부분

In [2]:
import sys
import subprocess

# try:
#     # 없는 모듈 import시 에러 발생
#     import pandas
# except:
#     # pip 모듈 업그레이드
#     subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'pip'])
#     # 에러 발생한 모듈 설치
#     subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'pandas'])
#     # 다시 import
#     import pandas

In [1]:
import os
from PIL import Image
from datetime import datetime
import shutil

try:
	import cv2
except:
    # pip 모듈 업그레이드
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'pip'])
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'opencv-python'])
    import cv2

try:
    from skimage.metrics import structural_similarity as ssim
except:
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'scikit-image'])
    from skimage.metrics import structural_similarity as ssim
    from skimage import io

try:
    import imutils
except:
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'imutils'])
    import imutils

try:
    import argparse
except:
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'argparse'])
    import argparse


ModuleNotFoundError: No module named 'cv2'

## 2. 비디오 인식

In [ ]:
## 비디오 경로
video_Path = input("비디오 경로를 입력해주세요: ")

## opencv 비디오 인식
video = cv2.VideoCapture(video_Path)

## 비디오가 없을 때
if not video.isOpened():
    print("Cannot open the vieo : ", video_Path)
    exit(0)

In [ ]:
## 비디오 정보
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = round(video.get(cv2.CAP_PROP_FPS))

## 디렉토리 생성
save_NAME = input("저장할 이름을 입력해주세요: ")
save_DIR = "./result/" + save_NAME
try:
    if not os.path.exists(save_DIR):
        os.makedirs(save_DIR)
except OSError:
    print("Error: Cannot Find SAVE_PATH", save_DIR)




## 3. 비디오 처리

In [ ]:
## Define Frame Counter
frameCounter = 0

## Video Pre-processing
while(video.isOpened()):
    ret, image = video.read()
    if (int(video.get(1)) % fps == 0):
        cv2.imwrite(save_DIR + "/frame%d.jpg" % frameCounter, image)
        print("Saved Frame Number: ", str(int(video.get(1))))
        frameCounter += 1
    if int(video.get(1)) == length:
        break

video.release()

## Remove Video
os.remove(video_Path)
print("Eliminate Video!!")


## 4. 중복된 사진 삭제

In [ ]:
## Take Parse
ap = argparse.ArgumentParser()
filenum = len(os.listdir(save_DIR))

## 기능 구현
for i in range(filenum - 1):
    Primary = save_DIR + "/frame" + str(i) + ".jpg"
    Secondary = save_DIR + "/frame" + str(i + 1) + ".jpg"
    
    args = vars(ap.parse_args())
    
    ## Load Image
    imageA = cv2.imread(Primary)
    imageB = cv2.imread(Secondary)
    
    ## Filtered Gray Scale
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    
    ## Saving Score
    (score, diff) = ssim(grayA, grayB, full=True)
    diff = (diff * 225).astype("uint8")
    
    ## If Score is larger than 0.87, then delete Primary
    if score >= 0.87:
        os.remove(Primary)

## 5. PDF 변환

In [ ]:
print("=== PDF Generator ===")

file_list = os.listdir(save_DIR)

img_list = []
img_path = save_DIR + "/" + file_list[0]
im_buf = Image.open(img_path)
cvt_rgb_0 = im_buf.convert('RGB')
for i in file_list:
    img_path = save_DIR + "/" + i
    im_buf = Image.open(img_path)
    cvt_rgb = im_buf.convert('RGB')
    img_list.append(cvt_rgb)

del img_list[0]

savefilepath = "./result/pdf" + save_NAME
try:
    if not os.path.exists(savefilepath):
        os.makedirs(savefilepath)
except OSError:
    print("Error: Cannot Find Select Directory: " + savefilepath)

cvt_rgb_0.save(savefilepath + '/' + save_NAME + '.pdf', save_all=True, append_images=img_list)


## 6. 사진 파일 삭제

In [ ]:
shutil.rmtree(save_DIR, ignore_errors=True)
print("사진 제거")
print("강의 요약 완료")